In [ ]:
pip install sentence-transformers

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#model import
sbert_model = SentenceTransformer('paraphrase-mpnet-base-v2')

#data import
train_df = pd.read_csv("../input/nlp-getting-started/train.csv")
test_df = pd.read_csv("../input/nlp-getting-started/test.csv")

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub('[0-9]', '', text)
    text = re.sub('#', '', text)   
    return text

train_df["clean_text"] = train_df["text"].apply(clean_text)
test_df["clean_text"] = test_df["text"].apply(clean_text)

In [ ]:
Training_Sentence_embeddings = sbert_model.encode(train_df.clean_text)
Test_Sentence_embeddings = sbert_model.encode(test_df.clean_text)

In [ ]:
skf = StratifiedKFold(n_splits=4, random_state=48, shuffle=True)
accuracy=[]
y=train_df['target']
for trn_idx, test_idx in skf.split(Training_Sentence_embeddings, y):
    X_tr,X_val=Training_Sentence_embeddings[trn_idx],Training_Sentence_embeddings[test_idx]
    y_tr,y_val=y.iloc[trn_idx],y.iloc[test_idx]
    model= LogisticRegression(max_iter=1000,C=3)
    model.fit(X_tr,y_tr)
    s = model.predict(X_val)
    accuracy.append(accuracy_score(y_val, s))
     

In [ ]:
mean_acc = np.mean(accuracy)*100
print("Cross Validated Training Set Accuracy: ", mean_acc)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
pred_valid_y = model.predict(X_val)
print("Confusion Matrix: ")
print(confusion_matrix(y_val, pred_valid_y ))
print("Classification Report: ")
print(classification_report(y_val, pred_valid_y ))

In [ ]:
FinalModel = LogisticRegression(max_iter=1000,C=3)
FinalModel.fit(Training_Sentence_embeddings,train_df['target'])

In [ ]:
sample_submission = pd.read_csv("../input/nlp-getting-started/sample_submission.csv")
sample_submission.target = FinalModel.predict(Test_Sentence_embeddings)
print(sample_submission.head(10))
sample_submission.to_csv("submission.csv", index=False)